# Data collection of soccer players

## Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import numpy as np
import pandas as pd
from tqdm import tqdm
import time 
import csv
import re

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
           "Accept-Language": "en-US,en;q=0.5"}

#Club Table

### id

In [ ]:
def player_id(link):
  try:
    player_id = (str(link).split('/'))[-1]
    return player_id
  except:
    print(link +" player id not found!")
    return np.nan

In [ ]:
PROXY_ADDRESS = "185.199.229.156"
PROXY_PORT =7492
PROXY_USERNAME = "zhudofyh"
PROXY_PASSWORD = "k1tm10gcjs3u"

proxy_credential = {"https": f"http://{PROXY_USERNAME}:{PROXY_PASSWORD}@{PROXY_ADDRESS}:{PROXY_PORT}"}

In [ ]:
headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

#get all club links for five countries

In [ ]:
country_url={'England':' https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1',
             'Spain' : 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1',
             'Italy':   'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1',
             'Germany':'https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1',
             'France': 'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1'}

In [ ]:
links=[]
clubs=[]
ids=[]
countries=[]
seasons=[]
years=np.arange(2015,2022) 


for country, url in country_url.items():
    for year in years:
        s_url=url +'/plus/?saison_id='+ str(year) 
        r = requests.get(s_url,headers =headers , proxies=proxy_credential)
        soup = BeautifulSoup(r.content, 'html.parser')
        time.sleep(20)
    
        table=soup.find('table',attrs={'class':'items'})
        trs=table.find_all('tr')  
        
        for tr in trs[2:]:
            td=tr.find('td',attrs={'class':'hauptlink no-border-links'}).find('a', href=True) 
            link='https://www.transfermarkt.com'+ td['href']
            name=td['title'] 
            id=link.split('https://www.transfermarkt.com/')[1].split('/')[3] 
            if link not in links:
                clubs.append(name)
                ids.append(id) 
                countries.append(country)
                links.append(link)
                seasons.append(year)

<bound method Tag.prettify of <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>ERROR: The request could not be satisfied</title>
</head><body>
<h1>403 ERROR</h1>
<h2>The request could not be satisfied.</h2>
<hr noshade="" size="1px"/>
Request blocked.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<br clear="all"/>
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<br clear="all"/>
<hr noshade="" size="1px"/>
<pre>
Generated by cloudfront (CloudFront)
Request ID: gK-FcYF_M2PS61xECTkA-uzCSHdkeetXiyfWNLWZw1VFcixe0o_nlA==
</pre>
<address>
</address>
</body></html>>


AttributeError: ignored

#Expenditure,Income,Over_all_balance

In [ ]:
links=[]
clubs=[]
ids=[]
countries=[]
seasons=[]
income_arrivals=[]
incomes=[]
overall_balance=[]
expend_arrivals=[]
expends=[]
years=np.arange(2015,2022) 


for country, url in country_url.items():
    for year in years:
        s_url=url +'/plus/?saison_id='+ str(year) 
        main_page= requests.get(s_url,headers=headers)
        soup = BeautifulSoup(main_page.content, 'html.parser')
        table=soup.find('table',attrs={'class':'items'})
        trs=table.find_all('tr') 
        for tr in trs[2:]:
            td=tr.find('td',attrs={'class':'hauptlink no-border-links'}).find('a', href=True) 
            link='https://www.transfermarkt.com'+ td['href']
            name=td['title'] 
            id=link.split('https://www.transfermarkt.com/')[1].split('/')[3] 
            clubs.append(name)
            ids.append(id) 
            countries.append(country)
            links.append(link)
          
            seasons.append(year)
            #tranfer table 
            page=requests.get(link,headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser') 
            transfer_record_box=soup.find_all('div', attrs={'data-viewport':'Transferbilanz'})[0] 
            income_td=transfer_record_box.find('table').find('tbody').find('tr',attrs={'class':'transfer-record__revenue'})
            income_arrival=income_td.find('td',attrs={'class':'zentriert test-class transfer-record__text'}).text
            income_arrivals.append(income_arrival)

            income=income_td.find('td',attrs={'class':'transfer-record__total transfer-record__total--positive'}).text.split('\n')[1].rstrip().lstrip()
            incomes.append(income) 

            expend_td=transfer_record_box.find('table').find('tbody').find('tr',attrs={'class':'transfer-record__expenses'})
            expend_arrival=expend_td.find('td',attrs={'class':'zentriert transfer-record__text'}).text 
            expend_arrivals.append(expend_arrival)

            expend=expend_td.find('td',attrs={'class':'transfer-record__total transfer-record__total--negative'}).text.split('\n')[1]
            expends.append(expend)
            try:
                overall_td=transfer_record_box.find('table').find('tfoot').find('tr').find('td',attrs={'class':'redtext rechts transfer-record__total'}).text.replace('\n','')
            except:
                try:
                    overall_td=transfer_record_box.find('table').find('tfoot').find('tr').find('td',attrs={'greentext rechts transfer-record__total'}).text.rstrip().lstrip()
                except:
                    overall_td=transfer_record_box.find('table').find('tfoot').find('tr').find('td',attrs={'rechts transfer-record__total'}).text.rstrip().lstrip().replace('+','')

                  
            overall_balance.append(overall_td)           
          

In [ ]:
clubs_id={}
clubs_id['Club']=clubs
clubs_id['id']=ids
clubs_id['Country']=countries
clubs_id['link']=links
clubs_id['Season']=seasons
clubs_id['income']=incomes 
clubs_id['expend']=expends
clubs_id['overall_balance']=overall_balance
clubs_id['income_arrival']=income_arrivals
clubs_id['expend_arrival']=expend_arrivals
df=pd.DataFrame(data=clubs_id) 
print(df.shape)
df.head()

(686, 10)


,Club,id,Country,link,Season,income,expend,overall_balance,income_arrival,expend_arrival
0,Chelsea FC,631,England,https://www.transfermarkt.com/fc-chelsea/start...,2015,€87.49m,€96.50m,€-9.01m,32,37
1,Manchester City,281,England,https://www.transfermarkt.com/manchester-city/...,2015,€67.44m,€208.47m,€-141.03m,23,20
2,Arsenal FC,11,England,https://www.transfermarkt.com/fc-arsenal/start...,2015,€2.50m,€26.50m,€-24.00m,11,13
3,Manchester United,985,England,https://www.transfermarkt.com/manchester-unite...,2015,€100.67m,€156.00m,€-55.33m,20,22
4,Liverpool FC,31,England,https://www.transfermarkt.com/fc-liverpool/sta...,2015,€90.55m,€126.50m,€-35.95m,24,23


In [ ]:
df.to_csv('over_all_balance.csv', index=False)